In [ ]:
!pip install openai langchain langchain-openai faiss-cpu python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Configura a chave de API (opcional, a biblioteca openai já faz isso se a variável de ambiente estiver definida)
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Inicializa o modelo de linguagem
# O gpt-3.5-turbo é uma ótima opção por ser rápido e barato
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Inicializa a memória para guardar o histórico da conversa
memory = ConversationBufferMemory()

# Cria a "Chain" de conversação, unindo o LLM e a memória
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True # verbose=True nos ajuda a ver o que está acontecendo por trás dos panos
)

In [ ]:
# Primeira interação: O assistente se apresenta e começa a qualificação
prompt_inicial = "Olá! Sou o assistente virtual da Welhome. Para começarmos, pode me dizer quantos imóveis você possui para alugar e onde eles se localizam?"
resposta_assistente = conversation_chain.predict(input=prompt_inicial)
print(resposta_assistente)

In [ ]:
# Segunda interação: O usuário responde, e o assistente continua a qualificação [cite: 53]
prompt_usuario_1 = "Tenho 2 apartamentos em São Paulo, na Vila Madalena."
resposta_assistente_2 = conversation_chain.predict(input=prompt_usuario_1)
print(resposta_assistente_2)

In [ ]:
# Terceira interação: O usuário responde à última pergunta [cite: 53]
prompt_usuario_2 = "Minha experiência é baixa, sempre aluguei por conta própria e tive alguns problemas."
resposta_assistente_3 = conversation_chain.predict(input=prompt_usuario_2)
print(resposta_assistente_3)

In [ ]:
from langchain.prompts import PromptTemplate

# Pega todo o histórico da conversa que está na memória
historico_conversa = memory.buffer_as_str

# Cria um template de prompt para a tarefa de sumarização
prompt_template_resumo = PromptTemplate(
    input_variables=["historico"],
    template="""
    Com base no histórico de conversa abaixo, gere um resumo estruturado para um vendedor humano.
    O resumo deve ser claro, conciso e conter as informações mais importantes do lead.
    Use o seguinte formato:
    - **Quantidade de Imóveis:** [Número de imóveis]
    - **Localização:** [Cidade e bairro]
    - **Experiência Prévia:** [Descrição da experiência]
    - **Principal Dor/Necessidade:** [Inferir a principal necessidade do lead a partir da conversa]

    Histórico da Conversa:
    {historico}
    """
)

# Cria uma nova "Chain" apenas para a tarefa de resumo
summarization_chain = prompt_template_resumo | llm

# Executa a chain de resumo, passando o histórico
resumo_para_vendedor = summarization_chain.invoke({"historico": historico_conversa})

print("--- RESUMO PARA O VENDEDOR ---")
print(resumo_para_vendedor.content)

In [ ]:
faq_welhome = """
Pergunta: Como a Welhome garante que o aluguel será pago em dia?
Resposta: A Welhome oferece a garantia de aluguel em dia. Mesmo que o inquilino atrase, nós garantimos o repasse do valor para o proprietário na data combinada, sem custos adicionais.

Pergunta: Quem é responsável pela manutenção do imóvel?
Resposta: Reparos estruturais são de responsabilidade do proprietário. Pequenas manutenções do dia a dia, como troca de lâmpadas, são do inquilino. Nossa plataforma ajuda a intermediar e orçar serviços de manutenção quando necessário.

Pergunta: Qual é o custo da taxa de administração da Welhome?
Resposta: Nossa taxa de administração é de 8% sobre o valor do aluguel. Essa taxa cobre a gestão completa do seu imóvel, incluindo divulgação, gestão de contratos, repasse garantido e suporte.

Pergunta: Como funciona a vistoria do imóvel?
Resposta: Realizamos uma vistoria profissional completa, com fotos e vídeos, antes da entrada do inquilino e após a sua saída. Isso garante que o imóvel seja devolvido nas mesmas condições em que foi entregue.
"""

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# 1. Dividir (Chunk): Quebra o texto do FAQ em pedaços menores
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([faq_welhome])

# 2. Vetorizar e Armazenar (Embed & Store): Cria os embeddings e armazena no FAISS [cite: 59]
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(docs, embeddings)

# 3. Criar um "Retriever": um objeto que sabe como buscar informações no vector_store
retriever = vector_store.as_retriever()

# 4. Criar o Prompt para o RAG: Instruí o LLM a usar o contexto recuperado
prompt_rag_template = PromptTemplate.from_template("""
Responda a pergunta do usuário com base apenas no contexto fornecido.
Se a resposta não estiver no contexto, diga "Não encontrei essa informação na minha base de dados.".

Contexto:
{context}

Pergunta: {input}
""")

# 5. Criar a "Chain" de RAG: Junta o prompt, o LLM e o retriever
question_answer_chain = create_stuff_documents_chain(llm, prompt_rag_template)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("Setup do RAG concluído. Pronto para receber perguntas.")

In [ ]:
# Pergunta cuja resposta ESTÁ no FAQ
pergunta_1 = "O que acontece se o meu inquilino não pagar o aluguel?"
resposta_1 = rag_chain.invoke({"input": pergunta_1})

print(f"Pergunta: {pergunta_1}")
print(f"Resposta: {resposta_1['answer']}\n")


# Pergunta cuja resposta também ESTÁ no FAQ
pergunta_2 = "Quanto custa o serviço de vocês?"
resposta_2 = rag_chain.invoke({"input": pergunta_2})

print(f"Pergunta: {pergunta_2}")
print(f"Resposta: {resposta_2['answer']}\n")


# Pergunta cuja resposta NÃO ESTÁ no FAQ
pergunta_3 = "Vocês administram imóveis comerciais?"
resposta_3 = rag_chain.invoke({"input": pergunta_3})

print(f"Pergunta: {pergunta_3}")
print(f"Resposta: {resposta_3['answer']}")